In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [ ]:
train = pd.read_csv('../input/train.csv')
test = pd.read_csv('../input/test.csv')
submit = pd.read_csv('../input/gender_submission.csv')

### Label Meaning

* Name: 姓名
* Age: 年齡
* Sex: 性別
* Fare: 票價
* Ticket_info: 票務資訊
* Cabin: 艙號
* Parch: 父母/小孩數
* SibSp: 兄弟/姊妹/配偶數
* Pclass: 乘票等級
* Embarked: 港口
* PassengerId: 乘客編號
* Survived: 是否存活

#### 自己加ㄉ
* Family_size: 家庭成員數量
* IsAlone: 是否單獨一人

In [ ]:
def mean_normalization(data):
    return (data - np.min(data)) / (np.max(data) - np.min(data))

In [ ]:
def preprocess(X_data): # train data total 891 entries
    # Sex preprocess -> one hot
    X_data = pd.concat([pd.get_dummies(X_data['Sex'], prefix = 'sex'), X_data], axis = 1)
    X_data.pop('Sex')
    # change Name label
    X_data['Name'] = X_data['Name'].str.split(', ', expand = True)[1].str.split('.', expand = True)[0]
    X_data['Name'] = X_data['Name'].replace(['Lady', 'Countess', 'Capt', 'Col', 'Don', 'Dr', 'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona'], 'Rare')
    X_data['Name'] = X_data['Name'].replace(['Mlle', 'Ms'], 'Miss')
    X_data['Name'] = X_data['Name'].replace('Mme', 'Mrs')
    X_data['Name'] = X_data['Name'].astype('category').cat.codes
    # modify Ticket_info label
    X_data['Ticket_info'] = X_data['Ticket'].apply(lambda x : x.replace('.',"").replace('/',"").strip().split(' ')[0] if not x.isdigit() else 'X')
    X_data['Ticket_info'] = X_data['Ticket_info'].astype('category').cat.codes
    X_data.pop('Ticket')
    # Cabin only 204
    X_data['Cabin'] = X_data['Cabin'].fillna('NoCabin')
    X_data['Cabin'] = X_data['Cabin'].apply(lambda x: x.split()[0].strip('1234567890'))
    X_data['Cabin'] = X_data['Cabin'].astype('category').cat.codes
    # Embarked 港口, only 889
    X_data = pd.concat([pd.get_dummies(X_data['Embarked'], prefix = 'embarked'), X_data], axis = 1)
    X_data.pop('Embarked')
    # age only 714, 依據姓名的稱謂決定年齡
    title_age_sum_dict = {}
    title_age_counter_dict = {}
    title_age_mean_dict = {}
    for title in X_data['Name']:
        title_age_sum_dict[title] = 0
        title_age_counter_dict[title] = 0
    for index, data in X_data.iterrows():
        if data['Name'] in X_data['Name']:
            if not pd.isna(data['Age']):
                title_age_sum_dict[data['Name']] += data['Age']
                title_age_counter_dict[data['Name']] += 1
    print(title_age_sum_dict)
    print(title_age_counter_dict)
    for title in X_data['Name']:
        title_age_mean_dict[title] = title_age_sum_dict[title] / title_age_counter_dict[title]
    print(title_age_mean_dict)
    mean = X_data['Age'].mean()
    for index, data in X_data.iterrows():
        if pd.isna(data['Age']):
            if pd.isna(title_age_mean_dict[data['Name']]):
                X_data.loc[index, 'Age'] = mean
            else:
                X_data.loc[index, 'Age'] = title_age_mean_dict[data['Name']]
    # add label Family_size by Parch and SibSp
    X_data['Family_size'] = X_data['Parch'] + X_data['SibSp'] + 1
    # add label IsAlone by Family_size
    X_data['IsAlone'] = 0
    X_data.loc[X_data['Family_size'] == 1, 'IsAlone'] = 1
    # pop PassengerId
    X_data.pop('PassengerId')
    # Name, Age, Fare, Ticket_info, Cabin, Family_size, Parch, SibSp, Pclass -> mean_normalization
    X_data['Name'] = mean_normalization(X_data['Name'])
    X_data['Fare'] = mean_normalization(X_data['Fare'])
    X_data['Age'] = mean_normalization(X_data['Age'])
    X_data['Ticket_info'] = mean_normalization(X_data['Ticket_info'])
    X_data['Cabin'] = mean_normalization(X_data['Cabin'])
    X_data['Family_size'] = mean_normalization(X_data['Family_size'])
    X_data['Parch'] = mean_normalization(X_data['Parch'])
    X_data['SibSp'] = mean_normalization(X_data['SibSp'])
    X_data['Pclass'] = mean_normalization(X_data['Pclass'])
    
    return X_data

In [ ]:
x_train = preprocess(train)
x_test = preprocess(test)

In [ ]:
# get y_train, y_test
y_train = x_train.pop('Survived')
y_test = submit.pop('Survived')

In [ ]:
x_train.head(20)

In [ ]:
x_test.head(1)

### 在這邊寫上你的 Deep Learning Model 吧~

In [ ]:
### Start Code Here

### End Code Here

如果你的 model 沒有使用 validation_split，記得把 `val_acc`、`val_loss` 那兩行**註解**掉。

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

# 繪製訓練 & 驗證的準確值
plt.subplot(2,1,1)
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc']) # <= 如果沒有 validation_split 這行記得註解掉

plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc = 'lower right')

# 繪製訓練 & 驗證的損失值
plt.subplot(2,1,2)
plt.title('Model loss')
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss']) # <= 如果沒有 validation_split 這行記得註解掉

plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc = 'upper right')
plt.show()
print("Loss: {}, Acc: {}".format(evaluate[0], evaluate[1]))

In [ ]:
predict = model.predict(x_test)

In [ ]:
predict = np.where(predict >= 0.5, 1, 0)

In [ ]:
predict_train = model.predict(x_train)
predict_train = np.where(predict_train >= 0.5, 1, 0)
counter = 0
for i in range(len(predict_train)):
    if predict_train[i][0] == y_train[i]:
        counter += 1
print(counter)
print(counter / len(predict_train))

print(predict)

In [ ]:
submit['Survived'] = predict
submit.to_csv('submission.csv', index = False)

In [ ]:
# import the modules we'll need
from IPython.display import HTML
import pandas as pd
import numpy as np
import base64

# function that takes in a dataframe and creates a text link to  
# download it (will only work for files < 2MB or so)
def create_download_link(df, title = "Download CSV file", filename = "data.csv"):  
    csv = df.to_csv(index = False)
    b64 = base64.b64encode(csv.encode())
    payload = b64.decode()
    html = '<a download="{filename}" href="data:text/csv;base64,{payload}" target="_blank">{title}</a>'
    html = html.format(payload = payload, title = title, filename = filename)
    return HTML(html)

# create a random sample dataframe
df = pd.DataFrame(np.random.randn(50, 4), columns = list('ABCD'))

# create a link to download the dataframe
create_download_link(submit)